In [1]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs
from dbio3 import to_db, load_data

# 뉴스목록, 날짜, id 수집

In [2]:
url1 = "https://fintech.or.kr/web/board/boardContentsList.do"

In [3]:
payload = dict(miv_pageNo=2, mode="W", board_id=6)

In [4]:
r1 = requests.get(url1, params=payload) 
print(r1.status_code) 
soup = bs(r1.content, 'lxml') 
soup 

200


<html><head><script language="javascript">
$(document).ready(function(){

	// 댓글수 노출여부 확인후 숨기기(미구현)
	
		$(".comment_cnt").hide();
	

	// 답변 노출여부 확인후 숨기기(미구현)
	
		$(".reply_status").hide();
	

	// 모바일 전용 페이지 숨기지
	//$(".mobile_list").hide();

	$(function() {

	});

	$("#searchtxt").keydown(function(key) {
		if (key.keyCode == 13) {
			search();
		}
	});
});
</script>
</head><body><div class="boardlist_top">
<ul class="box">
<li class="select">
<div class="optionbox">
<select id="searchkey" name="searchkey">
<option value="I">전체</option>
<option value="T">제목</option>
<option value="C">내용</option>
</select>
</div>
</li>
<li class="input">
<div class="inpbox"><input class="txt" id="searchtxt" name="searchtxt" placeholder="검색어 입력" title="검색어 입력" type="text" value=""/></div>
</li>
<li class="button">
<button class="btn3 bg_blue" onclick="search();" title="검색" type="button">검색</button>
<!-- <button type="button" class="btn3 bg_gray" title="재검색">재검색</button> -->
</li>
</ul>
</div>
<!--// list_t

In [5]:
len(soup.select("ul.list > li > a"))

15

web용은 테이블 안에

In [6]:
# 제목과 뉴스 id
soup.select("a.txtl")[0]['href'].split("'")[1]

'0579e2d50ca4449f97f5c21e6acf3d1c'

In [7]:
# 날짜
soup.select("td.last")[0].text.strip()

'2025-11-18'

# news list id와 날짜 모으기

In [8]:
content_id_list = []
for date, content_list in zip(soup.select("td.last"), soup.select("a.txtl")):
    date = date.text.strip()
    content_id = content_list['href'].split("'")[1]
    content_id_list.append((date, content_id))
content_id_list

[('2025-11-18', '0579e2d50ca4449f97f5c21e6acf3d1c'),
 ('2025-11-17', 'bc4b472316354894a61a9d7515fd585a'),
 ('2025-11-14', '06baae1d74e84020897b429e5908b67d'),
 ('2025-11-13', '4344b8eae1af467ab79269e3ccbe8727'),
 ('2025-11-12', 'f6d1b17da67748bbbd638a8498cab027'),
 ('2025-11-11', '65c5f16779c34033b575a702bad9ea15'),
 ('2025-11-10', '29a8c0a2afff4200b3846431fd3daeb8'),
 ('2025-11-07', '4bf5be7d0aff47bdabbd34bc2962ab50'),
 ('2025-11-06', '0fa0423c31b2439ba94e894e3897aada'),
 ('2025-11-05', 'bc1ef267eda14e3ea39e89936cc5854d'),
 ('2025-11-05', 'b08264a2396f4bd1aea6b1632ade7919'),
 ('2025-11-04', '0c07345c3b6149a983fd78ef5487ae96'),
 ('2025-11-03', '50f770f84cd44d078133ba3869256a22'),
 ('2025-10-31', '4383ccafc29440d885a80d9daef6f134'),
 ('2025-10-30', 'f4a8b125276d4d49bfeb7dcf711907ee')]

In [9]:
# newslist의 url, payload
url2 = "https://fintech.or.kr/web/board/boardContentsView.do"
payload2 = dict(miv_pageNo=2, mode="W", contents_id="0579e2d50ca4449f97f5c21e6acf3d1c", board_id=6)

In [10]:
r2 = requests.get(url2, params=payload2) 
print(r2.status_code) 
soup2 = bs(r2.content, 'lxml') 
soup2

200


<!DOCTYPE html>
<html lang="ko" xml:lang="ko">
<head>
<title>
		
		
		핀테크 포털 - 한국핀테크지원센터
		
		</title>
<meta content="핀테크 생태계 활성화로 금융의 혁신과 성장을 지원합니다.." name="description"/>
<meta content="핀테크 포털 - 한국핀테크지원센터" name="keywords"/>
<meta content="http://fintech.or.kr/images/preview_page.png" property="og:image"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=no" name="viewport"/>
<meta content="XpressEngine" name="Generator"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="o8yb702i79l2intg1e9ov2hyavzbji" name="facebook-domain-verification"/>
<!-- <link rel="shortcut icon" href="/images/web/favicon.ico" type="image/x-icon"> -->
<!-- <link rel="icon" href="/images/web/favicon.ico" type="image/x-icon"> -->
<link href="/css/web/default.css?ver=20200916" rel="stylesheet" type="text/css"/>
<link href="/css/web/reset.css?ver=20210629" rel="stylesheet" t

In [11]:
# 세부링크
soup2.select("tbody td > a")[0]['href']

'https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=fe915b15bcca4d11b46d83b70a41207d&menu_id=6300'

In [12]:
# 뉴스제목
soup2.select("tbody td > a")[0].text

'[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Academy’참여자 모집(~선착순 마감)'

# 코드 모으기

In [13]:
for td in soup2.select("tbody td > a"):
    if "https://n.news.naver.com" in td['href'] or "https://fintech.or.kr" in td['href']:
        print(td.text)
        print(td['href'])

[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Academy’참여자 모집(~선착순 마감)
https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=fe915b15bcca4d11b46d83b70a41207d&menu_id=6300
[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생 모집(상시)
https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=e04c846efa67421db116e5533c0d8bd2&menu_id=6300
[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) 참여기업 모집 (~예산 소진 시까지)
https://fintech.or.kr/web/board/boardContentsView.do?board_id=1&contents_id=abc17a4ab12d4c79ae94d177994b4d57&menu_id=6100
150조 '국민성장펀드' 본격 가동…이억원 "금융권 역량 총동원"
https://n.news.naver.com/mnews/article/015/0005212305?sid=101
금감원, '포인트 자동사용 서비스' 모든 카드사로 확대
https://n.news.naver.com/mnews/article/003/0013604464?sid=101
금감원, '모험자본 워크숍' 개최…“발행어음·기업금융 동일 본부 지양”
https://n.news.naver.com/mnews/article/011/0004556759?sid=101
금융위 “빚투, 엄격한 리스크 관리 필요… 신용거래 매일 점검”
https://n.news.naver.com/mnews/article/020/0003675129?sid=101
"지문으로 카드결제 인증 세계 최초"…코나아이 상용화 성공
https://n.news.naver.

In [14]:
# 핀테크 뉴스 리스트, 날짜, content_id
content_id_list = []
for page in range(1,20):
    url1 = "https://fintech.or.kr/web/board/boardContentsList.do"
    payload = dict(miv_pageNo=page, mode="W", board_id=6)
    r1 = requests.post(url1, data=payload) 
    print(r1.status_code) 
    soup = bs(r1.content, 'lxml') 

    for date, content_list in zip(soup.select("td.last"), soup.select("a.txtl")):
        date = date.text.strip()
        content_id = content_list['href'].split("'")[1]
        content_id_list.append((date, content_id))
    time.sleep(2)
content_id_list
        
result = {}
    
for idx, (date, content_id) in enumerate(content_id_list):
    print(f"{idx+1}/{len(content_id_list)} 수집중", end="\r")
    url2 = "https://fintech.or.kr/web/board/boardContentsView.do"
    payload2 = dict(miv_pageNo=1, mode="W", contents_id=content_id, board_id=6, searchkey="l")
    r2 = requests.get(url2, params=payload2) 
    print(r2.status_code) 
    soup2 = bs(r2.content, 'lxml') 

    for td in soup2.select("tbody td > a"):
        if "https://n.news.naver.com" in td['href'] or "https://fintech.or.kr" in td['href']:
            result.setdefault('date', []).append(date)
            result.setdefault('제목', []).append(td.text)
            result.setdefault('뉴스링크', []).append(td['href'])
            
df = pd.DataFrame(result)
df

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
20085 수집중
20085 수집중
20085 수집중
20085 수집중
20085 수집중
20085 수집중
20085 수집중
20085 수집중
20085 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집중
200285 수집

,date,제목,뉴스링크
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...
...,...,...,...
11771,2024-12-17,반짝 효과 그친 中 경기 부양… 소비·투자 일제히 둔화,https://n.news.naver.com/mnews/article/366/000...
11772,2024-12-17,"월가 ""달러,내년 중반에 정점 찍고 하락세로 돌아설 것""",https://n.news.naver.com/mnews/article/015/000...
11773,2024-12-17,트럼프 “규제 1개 신설시 10개 철폐…모든 카드 갖고 관세 협상”,https://n.news.naver.com/mnews/article/018/000...
11774,2024-12-17,"ECB 총재 ""물가 하방 위험…금리 계속 인하""(종합)",https://n.news.naver.com/mnews/article/001/001...


# 세부링크 들어가서 뉴스 본문 수집

In [15]:
df2 = df[df['date'] > "2025-01-01"].copy()

In [16]:
df2

,date,제목,뉴스링크
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...
...,...,...,...
11271,2025-01-02,"""당장 돈 안되고 성능향상 기대 못 미쳐도""… 세계는 AI인프라 `영끌`",https://n.news.naver.com/mnews/article/029/000...
11272,2025-01-02,트럼프 2.0시대 중국…미 동맹국 이탈 기대하며 버티기[다시 만난 트럼프②],https://n.news.naver.com/mnews/article/032/000...
11273,2025-01-02,'美 51번째 주' 모욕 당했는데…트럼프에 찍소리 못하는 이유 [김리안의 에네르기파...,https://n.news.naver.com/mnews/article/015/000...
11274,2025-01-02,[산업은행] 2025년 상반기 KDB NextONE 참여 스타트업 모집(서울/부산)...,https://fintech.or.kr/web/board/boardContentsV...


In [17]:
df2['뉴스링크'][0]

'https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=ba9a817cc8434910a4c6f47ff2e19099&menu_id=6300'

In [18]:
r3 = requests.get(df2['뉴스링크'][0])
soup3 = bs(r3.content, 'lxml')
soup3.select_one("div.content").text

'[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집(~12/8까지) \xa0  □ 프로그램 개요 - 국내 최대 핀테크 전문 지원기관인 한국핀테크지원센터와 국내 대기업(카카오, KT클라우드등) 부트캠프를 설계.운영해온 국내 대표 에듀테크 전문기관 구름(goorm)이 함께하는 핀테크 및 디지털 금융 산업 진출을\xa0목표로 하는 예비 실무자들을 위해 기획된 6개월 집중형 K-Digital Training 입니다.\ufeff \xa0  □ 신청 홈페이지 - (공식 신청 홈페이지) https://fintech-internship-course.goorm.io/ - (상세 안내 페이지)  https://www.notion.so/goormkdx/x-goorm-293c0ff4ce318024bea5ddcba23965d6 \xa0  □ 핀테크 인턴십 코스 과정 핵심 Point ① 핀테크 기업연계 현장형(인턴) 프로젝트 참여 : 한국핀테크지원센터의 협력기업과 현업 프로젝트 직접 참여 \xa0  ② 기업연계 매칭기업 서류전형 면제 : 프로젝트 진행 기업의 경우 채용전형 중 서류전형 면제 \xa0  ③ 우수 수료생 대상 인턴십 연계 : 수료자 중 우수 인재에게 협력기업 인턴십 또는 채용 연계 기회 제공 \xa0  ④ 핀테크 일자리 매칭 플랫폼 서비스 지원 : 수료 후 최대 2년간 취업 매칭, 채용 공고 연계, 1:1 커리어 컨설팅 제공 \xa0  ⑤ 현직자 멘토링 & 금융·핀테크 기업 및 기관 현장 견학 : 업계 선배 멘토링, 핀테크 기업 견학, 금융기관 방문 등 실무 인사이트 제공 \xa0  □ 주요 일정  구분일정비고서류접수2025.11.24(월) ~ 12.8(월)마감일 자정까지 서류 접수 가능발표2025.11.24(월) ~ 12.8(월)서류 제출 후 순차적 합격 통지교육기간2025.12.18(목) ~ 2026.06.18(금) 6개월 진행 (총 960시간) \xa0  □ 모집 과정 및 인원 과정명주요내용모집정원(명)핀테크 서비스 기획시장 분석, 

In [19]:
r3 = requests.get(df2['뉴스링크'][6])
soup3 = bs(r3.content, 'lxml')
soup3.select_one("#dic_area").text

'\n\n\n\n\n[서울=뉴시스]이지민 기자 = 증권사의 전산사고가 끊이지 않고 발생하는 가운데 금융감독원이 자본시장 IT 및 정보보안에 대한 엄정한 검사·제재 기조를 재차 강조했다.금감원은 25일 서울 여의도 금융투자협회에서 증권사 IT·정보보안 임원 및 유관기관 관계자 약 150명을 대상으로 \'자본시장 IT 및 정보보안 안전성 제고\' 정례 간담회를 개최했다. 금감원은 이날 간담회에서 증권업계에 자율점검 등 IT·정보보안 종합대책을 철저한 준수하고, 전산사고 발생 시 적극적인 투자자 피해보상과 재발 방지 대책 마련을 주문했다. 특히 프로그램 오류 등 빈번한 사고 유형에 대해 원인 분석과 대책수입을 요청했다.또 지난 3월 발생한 한국거래소(KRX) 전산사고를 계기로 관계기관 태스크포스(TF)를 통해 마련한 \'자본시장 통합 업무지속성계획(BCP)\'의 주요 내용을 공유했다. 통합 BCP는 KRX와 대체거래소인 넥스트레이드(NXT) 가운데 한 곳에 장애가 발생하더라도 투자자 거래가 중단되지 않도록 대응 체계를 표준화한 것이 골자다. 특정 집행시장에 일시적 장애가 발생할 경우, KRX와 NXT는 비상연락망을 구축해 증권사, 투자자 등에 즉시 비상 상황을 알려야 한다. 증권사는 장애 발생 통지를 받는 즉시 투자자에게 장애 상황과 주문 집행 프로세스를 안내해야 한다. 장애가 장시간 이어지면 KRX·NXT는 회원사에 거래정지 사실을 통지하고, 정상적으로 작동하는 다른 시장으로 거래가 몰릴 수 있도록 호가 접수 거부 등 통제 조치를 시행한다. 증권사는 SOR(주문배분시스템)을 활용해 정상 집행 시장으로 주문을 전송해야 한다. 아울러 이 같은 주문집행 방식을 각 사의 BCP 문서와 최선집행기준 설명서에 명확하게 기재하도록 했다.금감원 관계자는 "증권업계 등과의 긴밀하고 지속적인 소통과 협업을 통해 전자금융거래 안전성 제고 방안을 원활히 이행하는 한편, 자본시장 IT 인프라의 안전성 제고 및 투자자 보호를 위해 보다 적극적인 대응을 지속 추진해 나갈 예정"이라고 전했다

In [20]:
df2['뉴스본문'] = ""
df2

,date,제목,뉴스링크,뉴스본문
0,2025-11-26,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...,https://fintech.or.kr/web/board/boardContentsV...,
1,2025-11-26,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...,
2,2025-11-26,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...,
3,2025-11-26,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...,
4,2025-11-26,"""부산을 해양금융 허브로""…금감원·부산시·해진공·조선업계 총집결",https://n.news.naver.com/mnews/article/008/000...,
...,...,...,...,...
11271,2025-01-02,"""당장 돈 안되고 성능향상 기대 못 미쳐도""… 세계는 AI인프라 `영끌`",https://n.news.naver.com/mnews/article/029/000...,
11272,2025-01-02,트럼프 2.0시대 중국…미 동맹국 이탈 기대하며 버티기[다시 만난 트럼프②],https://n.news.naver.com/mnews/article/032/000...,
11273,2025-01-02,'美 51번째 주' 모욕 당했는데…트럼프에 찍소리 못하는 이유 [김리안의 에네르기파...,https://n.news.naver.com/mnews/article/015/000...,
11274,2025-01-02,[산업은행] 2025년 상반기 KDB NextONE 참여 스타트업 모집(서울/부산)...,https://fintech.or.kr/web/board/boardContentsV...,


In [32]:
for idx, link in enumerate(df2['뉴스링크'][:10]):
    print(f"{idx+1}/{len(df2['뉴스링크'][:10])} 수집중", end="\r")
    r4 = requests.get(link)
    soup4 = bs(r4.content, 'lxml')
    if "https://fintech.or.kr" in link:
        r3 = requests.get(link)
        soup3 = bs(r3.content, 'lxml')
        article = soup3.select_one("div.content").text
        df2.loc[idx, '뉴스본문'] = article
    elif "https://n.news.naver.com" in link:
        r4 = requests.get(link)
        soup4 = bs(r4.content, 'lxml')
        article = soup4.select_one("#dic_area").text
        df2.loc[idx, '뉴스본문'] = article
    to_db("fintech_news", "news_articles", df2.loc[[idx], :])
    time.sleep(3)
df2

fintech_news 데이터베이스 확인/생성 완료


OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'date' in 'field list'")
[SQL: INSERT INTO news_articles (date, `제목`, `뉴스링크`, `뉴스본문`) VALUES (%(date)s, %(제목)s, %(뉴스링크)s, %(뉴스본문)s)]
[parameters: {'date': '2025-11-26', '제목': "[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모집(~12.8.)", '뉴스링크': 'https://fintech.or.kr/web/board/boardContentsView.do?board_id=3&contents_id=ba9a817cc8434910a4c6f47ff2e19099&menu_id=6300', '뉴스본문': '[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집(~12/8까지) \xa0  □ 프로그램 개요 - 국내 최대 핀테크 전문 지원기관인 한국핀테크지원센터와 국내 대기업(카카오, KT클라우드등) 부트캠프를 설계.운영해온 국내 대표 에듀테크 ... (1353 characters truncated) ...  자    ③ 주중 오전 9시 ~ 오후 6시, 하루 8시간 수업에 성실히 참여 가능한 자  ※ 위 조건을 모두 충족해야 참여가 가능합니다. \xa0  □ 문의사항 - 한국핀테크지원센터 글로벌사업부 교육 담당자 - 이메일: kgm1012@fintech.or.kr\xa0'}]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [22]:
df2.to_csv("./data/fintech_news2.csv")

In [23]:
df2 = pd.read_csv("./data/fintech_news2.csv", index_col=0)

In [24]:
!pip install sqlalchemy pymysql

In [25]:
!pip install cryptography

In [26]:
from dbio3 import to_db, load_data

In [27]:
to_db("fintech_news", "news_articles", df2)

fintech_news 데이터베이스 확인/생성 완료
fintech_news.news_articles 데이터 저장 완료


In [ ]:
w2v_model = Word2Vec(sentences=all_tokenized, vector_size=100, window=5, min_count=2, workers=4, sg=1)

In [31]:
df3 = load_data("fintech_news", "news_articles")
df3

fintech_news 데이터베이스 확인/생성 완료


,0
0,2025-11-26
1,[한국핀테크지원센터X구름] K-디지털트레이닝 '핀테크 인턴십 코스' 4기 훈련생 모...
2,https://fintech.or.kr/web/board/boardContentsV...
3,[한국핀테크지원센터x구름]「K-디지털 트레이닝 핀테크 인턴십 코스」4기 훈련생 모집...
